In [ ]:
!yes | wget https://zenodo.org/records/5849371/files/marmoset-dlc-2021-05-07.zip -O marmoset-train.zip
!yes | unzip marmoset-train.zip -d .
!mv marmoset-dlc-2021-05-07 marmoset-train
!rm -r ./__MACOSX; rm -r **/__MACOSX
!mkdir marmoset-train/labels
!mkdir marmoset-train/labels/train
!mkdir marmoset-train/labels/val
# !yes | wget https://zenodo.org/records/8437121/files/marmoset-dlc-2021-05-07.zip -O marmoset-test.zip
# !yes | unzip marmoset-test.zip -d .
# !mv marmoset-dlc-2021-05-07 marmoset-test

--2024-09-03 05:44:41--  https://zenodo.org/records/5849371/files/marmoset-dlc-2021-05-07.zip
Resolving zenodo.org (zenodo.org)... 188.184.98.238, 188.184.103.159, 188.185.79.172, ...
Connecting to zenodo.org (zenodo.org)|188.184.98.238|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1924072708 (1.8G) [application/octet-stream]
Saving to: ‘marmoset-train.zip’

marmoset-train.zip   92%[=================>  ]   1.66G  18.0MB/s    eta 7s     

In [ ]:
## transform the json file to YOLO format
import json
import os
from collections import defaultdict
from pathlib import Path
import numpy as np
from tqdm import tqdm

def normalize_keypoints(keypoints, img_width, img_height, precision=6):
    normalized = []

    for i in range(0, len(keypoints), 3):
        x = max(0, min(1, round(keypoints[i] / img_width, precision)))
        y = max(0, min(1, round(keypoints[i+1] / img_height, precision)))
        v = keypoints[i+2]
        normalized.extend([x, y, v])

    return normalized

def compute_bounding_box(keypoints):
    points = [keypoints[i:i+3] for i in range(0, len(keypoints), 3) if keypoints[i+2] != 0]
    x_coords, y_coords = zip(*[(p[0], p[1]) for p in points])

    min_x, max_x = min(x_coords), max(x_coords)
    min_y, max_y = min(y_coords), max(y_coords)

    return [min_x, min_y, max_x - min_x, max_y - min_y]

def process_annotations(data, train_save_path, val_save_path, root_dir):
    id_map = {category['id']: i for i, category in enumerate(data['categories'])}
    images = {str(x['id']): x for x in data['images']}
    imgToAnns = defaultdict(list)

    for ann in data['annotations']:
        imgToAnns[ann['image_id']].append(ann)

    train_output_file = Path(root_dir, "train.txt").open('w')
    val_output_file = Path(root_dir, "val.txt").open('w')

    for img_id, anns in tqdm(imgToAnns.items(), desc="Processing Annotations"):
        img = images[str(img_id)]
        h, w, filename = img['height'], img['width'], img['file_name']
        actual_filename = filename.split("_")[-1]
        bboxes = []

        for ann in anns:
            keypoints = normalize_keypoints(ann['keypoints'], w, h)
            bbox = compute_bounding_box(keypoints)
            bbox[:2] = [x + y / 2 for x, y in zip(bbox[:2], bbox[2:])]  # Convert top-left to center
            cls = id_map[ann['category_id']]
            bbox = [cls] + bbox + keypoints

            if bbox not in bboxes:
                bboxes.append(bbox)

        save_path = val_save_path if "refinement" in filename else train_save_path
        with (Path(save_path) / actual_filename).with_suffix('.txt').open('w') as file:
            for bbox in bboxes:
                file.write(' '.join(map(str, bbox)) + '\n')

        if "refinement" in filename:
            val_output_file.write(f"./images/val/{actual_filename}\n")
        else:
            train_output_file.write(f"./images/train/{actual_filename}\n")

    val_output_file.close()
    train_output_file.close()

def main():
    print("Starting processing...")
    ROOT_DIR = "marmoset-train"
    JSON_FILE = "marmoset-train/training-datasets/iteration-0/UnaugmentedDataSet_marmosetMay7/dlc_shuffle1_train.json"
    TRAIN_SAVE_PATH = os.path.join(ROOT_DIR, "labels/train")
    VAL_SAVE_PATH = os.path.join(ROOT_DIR, "labels/val")

    with open(JSON_FILE, 'r') as file:
        data = json.load(file)

    process_annotations(data, TRAIN_SAVE_PATH, VAL_SAVE_PATH, ROOT_DIR)

if __name__ == "__main__":
    main()


Starting processing...


Processing Annotations: 100%|██████████| 3964/3964 [00:00<00:00, 6157.07it/s]


In [ ]:
# # clean up
!rm -rf marmoset-train/training-datasets

# create image folders
!mkdir marmoset-train/images
!mkdir marmoset-train/images/train
!mkdir marmoset-train/images/val

# move images to correct folders
!mv marmoset-train/labeled-data/reachingvideo1/* marmoset-train/images/train
!mv marmoset-train/labeled-data/refinement1/* marmoset-train/images/val
!rm -r marmoset-train/labeled-data
!mkdir datasets
!mv marmoset-train ./datasets/marmoset-train

mkdir: cannot create directory ‘marmoset-train/images’: File exists
mkdir: cannot create directory ‘marmoset-train/images/train’: File exists
mkdir: cannot create directory ‘marmoset-train/images/val’: File exists
mv: cannot stat 'marmoset-train/labeled-data/reachingvideo1/*': No such file or directory
mv: cannot stat 'marmoset-train/labeled-data/refinement1/*': No such file or directory
rm: cannot remove 'marmoset-train/labeled-data': No such file or directory


In [ ]:
import yaml

data = {
    "path": "./marmoset-train",
    "train": "train.txt",
    "val": "val.txt",
    "kpt_shape": [15, 3],
    "flip_idx": [0, 3, 2, 1, 6, 7, 4, 5, 9, 8, 11, 10, 12, 13, 14],
    "names": {
        "0": "marmoset"
    }
}

with open("marmoset-train.yaml", "w") as file:
    yaml.dump(data, file, default_flow_style=None, sort_keys=False)

In [ ]:
!pip install ultralytics
from ultralytics import YOLO

model = YOLO("yolov8n-pose.pt")

train_path = 'marmoset-train.yaml'
model.train(data=train_path, epochs=10)

Ultralytics YOLOv8.2.86 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=pose, mode=train, model=yolov8n-pose.pt, data=marmoset-train.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_box

100%|██████████| 755k/755k [00:00<00:00, 77.7MB/s]


Overriding model.yaml kpt_shape=[17, 3] with kpt_shape=[15, 3]

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1 

100%|██████████| 6.25M/6.25M [00:00<00:00, 88.6MB/s]


AMP: checks passed ✅


...

                   all        326        608      0.784      0.704       0.77      0.431      0.113     0.0707     0.0265    0.00331



10 epochs completed in 0.265 hours.
Optimizer stripped from runs/pose/train2/weights/last.pt, 6.7MB
Optimizer stripped from runs/pose/train2/weights/best.pt, 6.7MB

Validating runs/pose/train2/weights/best.pt...
Ultralytics YOLOv8.2.86 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n-pose summary (fused): 187 layers, 3,248,438 parameters, 0 gradients, 9.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:12<00:00,  1.14s/it]


                   all        326        608      0.784      0.704       0.77      0.432      0.113     0.0707     0.0264     0.0033
Speed: 0.4ms preprocess, 3.5ms inference, 0.0ms loss, 5.8ms postprocess per image
Results saved to runs/pose/train2


ultralytics.utils.metrics.PoseMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7eb20334c760>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(P)', 'F1-Confidence(P)', 'Precision-Confidence(P)', 'Recall-Confidence(P)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    

In [ ]:
!mv /content/runs/pose/train/weights/best.pt /content/marmo.pt

/content
settings_version: 0.0.5
datasets_dir: /content/datasets
weights_dir: weights
runs_dir: runs
uuid: 569f3ba64b326db489132663f79cd37279811de477381b83ac131e6cdd129cbb
sync: true
api_key: ''
openai_api_key: ''
clearml: true
comet: true
dvc: true
hub: true
mlflow: true
neptune: true
raytune: true
tensorboard: true
wandb: true
vscode_msg: true
